### Project 5

#### Goal 1

For this goal, you are given a number of CSV files, each of which have their first row with the field name.

You goal is to create a context manager that you can use to produce the data from each file in a named tuple with field names corresponding to the  header row field names.

You should use the `csv` module's `reader` function to help with parsing the data.

Your context manager should be generic in the sense that it should just need the file name, no other configuration or hardcoded functionality is required. You do not need to worry about data types for this goal - just return every field as a string.

In addition, your context manager should produce lazy iterators.

Implement this using a class that implements the context manager protocol

#### Goal 2

The goal is to reproduce the work you did in Goal 1, but using a generator function and the `contextlib` `contextmanager` decorator.

### Notes

The files included with this project are:
* `cars.csv`
* `personal_info.csv`

In addition you might find the following useful.

##### Reading and rewinding data from a File

The file object supports reading data by specifying the amount of data we want to read, and repositioning the "read head" using the `seek` function.

Let's take a look:

In [1]:
with open('cars.csv') as f:
    print('---', f.read(100))
    print('---', f.read(100))
    f.seek(0)
    print('---', f.read(100))
    

--- Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin
Chevrolet Chevelle Malibu
--- ;18.0;8;307.0;130.0;3504.;12.0;70;US
Buick Skylark 320;15.0;8;350.0;165.0;3693.;11.5;70;US
Plymouth 
--- Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin
Chevrolet Chevelle Malibu


As you can see, we could read the file calling the `read` method - this reads data and advances the "read head". We can then rewind and start again - either reading directly, or even just iterating through the rows using the iterator:

In [2]:
from itertools import islice

with open('cars.csv') as f:
    print('---', f.read(100))
    print('---', f.read(100))
    print('--------------------')
    print('rewinding to 0...')
    f.seek(0)
    for row in islice(f, 5):
        print(row, end='')

--- Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin
Chevrolet Chevelle Malibu
--- ;18.0;8;307.0;130.0;3504.;12.0;70;US
Buick Skylark 320;15.0;8;350.0;165.0;3693.;11.5;70;US
Plymouth 
--------------------
rewinding to 0...
Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin
Chevrolet Chevelle Malibu;18.0;8;307.0;130.0;3504.;12.0;70;US
Buick Skylark 320;15.0;8;350.0;165.0;3693.;11.5;70;US
Plymouth Satellite;18.0;8;318.0;150.0;3436.;11.0;70;US
AMC Rebel SST;16.0;8;304.0;150.0;3433.;12.0;70;US


##### Sniffing the CSV dialect

The dialect of a CSV file refers to some of the specifics used to define data in a CSV file. The separators can be different (for example some failes use a comma, some use a semi-colon, some use a tab, etc).

Also, as we have seen before, a field is also sometimes delimited using quotes, or double quotes, or maybe some entirely different character.

When we have to deal with files that may be encoded using different dialects it can require quite a bit of work to determine what those specifics are. This is were the `Sniffer` class from the `csv` module can be useful. By providing it a sample fo the CSV file, it can analyze it and determine a best guess as to the specific dialect that was used. We can then use that dialect when we use the `csv.reader` function.

Let's see how to use it with one of our files: `personal_info.csv`:

In [3]:
import csv

with open('personal_info.csv') as f:
    sample = f.read(2000)
    dialect = csv.Sniffer().sniff(sample)
print(vars(dialect))

{'__module__': 'csv', '_name': 'sniffed', 'lineterminator': '\r\n', 'quoting': 0, '__doc__': None, 'doublequote': False, 'delimiter': ',', 'quotechar': '"', 'skipinitialspace': False}


We can now use this dialect to open the csv reader:

In [4]:
with open('personal_info.csv') as f:
    reader = csv.reader(f, dialect)
    for row in islice(reader, 5):
        print(row)

['ssn', 'first_name', 'last_name', 'gender', 'language']
['100-53-9824', 'Sebastiano', 'Tester', 'Male', 'Icelandic']
['101-71-4702', 'Cayla', 'MacDonagh', 'Female', 'Lao']
['101-84-0356', 'Nomi', 'Lipprose', 'Female', 'Yiddish']
['104-22-0928', 'Justinian', 'Kunzelmann', 'Male', 'Dhivehi']
